# Chapter 08: 도메인 적응 SFT

## 1. 학습 목표

* 도메인 특화 데이터셋(금융, 의료, 코딩)의 특성을 이해한다.
* 시스템 프롬프트를 활용하여 모델에 전문가 페르소나를 부여한다.
* 금융 데이터셋을 사용하여 전문 지식을 갖춘 모델을 학습한다.
* 도메인 적응 시 고려해야 할 전략(용어 일관성, 면책 조항 등)을 학습한다.

## 2. 도메인 적응(Domain Adaptation)이란?

일반적인 LLM은 위키피디아나 웹 문서를 통해 넓고 얕은 지식을 가지고 있다. 하지만 특정 분야의 전문가처럼 행동하려면 해당 분야의 **전문 데이터**를 집중적으로 학습해야 한다.

* **일반 모델**: "주식 시장이 뭐야?" -> 일반적인 정의 설명
* **금융 특화 모델**: "주식 시장이 뭐야?" -> 자본 조달 기능, 유동성 공급, 가격 발견 기능 등 전문적 관점에서 설명

## 3. 도메인 데이터셋 준비

각 분야별로 대표적인 오픈소스 데이터셋들이 존재한다. 이번 실습에서는 금융 분야의 `gbharti/finance-alpaca`를 사용한다.

In [1]:
from datasets import load_dataset

# 도메인별 대표 데이터셋 (참고용)
domain_datasets = {
    "금융": "gbharti/finance-alpaca",
    "의료": "medalpaca/medical_meadow_medical_flashcards",
    "코딩": "TokenBender/code_instructions_122k_alpaca_style"
}

# 1. 금융 데이터셋 로드
print("금융 데이터셋 로딩 중...")
dataset = load_dataset("gbharti/finance-alpaca", split="train[:1000]")

print(f"데이터 개수: {len(dataset)}")
print(f"샘플 데이터:\n{dataset[0]}")

금융 데이터셋 로딩 중...
데이터 개수: 1000
샘플 데이터:
{'instruction': 'For a car, what scams can be plotted with 0% financing vs rebate?', 'input': '', 'output': "The car deal makes money 3 ways. If you pay in one lump payment. If the payment is greater than what they paid for the car, plus their expenses, they make a profit. They loan you the money. You make payments over months or years, if the total amount you pay is greater than what they paid for the car, plus their expenses, plus their finance expenses they make money. Of course the money takes years to come in, or they sell your loan to another business to get the money faster but in a smaller amount. You trade in a car and they sell it at a profit. Of course that new transaction could be a lump sum or a loan on the used car... They or course make money if you bring the car back for maintenance, or you buy lots of expensive dealer options. Some dealers wave two deals in front of you: get a 0% interest loan. These tend to be shorter 12 months vs 

## 4. 시스템 프롬프트 설계 (페르소나 주입)

도메인 적응의 핵심은 모델에게 **"당신은 전문가입니다"**라고 최면을 거는 것이다. 이를 위해 데이터 포맷팅 단계에서 시스템 프롬프트를 추가한다.

In [2]:
def format_domain_instruction(example, domain="금융"):
    """
    도메인별 시스템 프롬프트를 적용하여 데이터를 포맷팅하는 함수다.
    """
    # 도메인별 시스템 프롬프트 정의
    system_prompts = {
        "금융": "당신은 금융 전문가 AI 어시스턴트이다. 질문에 대해 전문적이고 분석적인 답변을 제공하라.",
        "의료": "당신은 의료 정보 AI 어시스턴트이다. 의학적 사실에 기반하여 답변하되, 반드시 전문가와의 상담을 권유하라.",
        "법률": "당신은 법률 전문가 AI 어시스턴트이다. 관련 법령과 판례에 기반하여 답변하되, 법적 효력이 없음을 명시하라."
    }

    instruction = example['instruction']
    input_text = example.get('input', '')
    output = example['output']

    # 해당 도메인의 시스템 프롬프트 선택
    sys_prompt = system_prompts.get(domain, "당신은 유용한 AI 어시스턴트이다.")

    # 프롬프트 구성
    if input_text:
        text = f"""### System:
{sys_prompt}

### Instruction:
{instruction}

### Input:
{input_text}

### Response:
{output}"""
    else:
        text = f"""### System:
{sys_prompt}

### Instruction:
{instruction}

### Response:
{output}"""

    return {"text": text}

# 금융 도메인으로 포맷팅 적용
formatted_dataset = dataset.map(lambda x: format_domain_instruction(x, "금융"))

print("\n[포맷팅된 데이터 샘플]")
print(formatted_dataset[0]['text'])


[포맷팅된 데이터 샘플]
### System:
당신은 금융 전문가 AI 어시스턴트이다. 질문에 대해 전문적이고 분석적인 답변을 제공하라.

### Instruction:
For a car, what scams can be plotted with 0% financing vs rebate?

### Response:
The car deal makes money 3 ways. If you pay in one lump payment. If the payment is greater than what they paid for the car, plus their expenses, they make a profit. They loan you the money. You make payments over months or years, if the total amount you pay is greater than what they paid for the car, plus their expenses, plus their finance expenses they make money. Of course the money takes years to come in, or they sell your loan to another business to get the money faster but in a smaller amount. You trade in a car and they sell it at a profit. Of course that new transaction could be a lump sum or a loan on the used car... They or course make money if you bring the car back for maintenance, or you buy lots of expensive dealer options. Some dealers wave two deals in front of you: get a 0% interest loan. These t

## 5. 모델 로드 및 학습 설정 (QLoRA)

이전 챕터에서 검증된 QLoRA 설정을 그대로 사용하여 효율적으로 학습한다. 베이스 모델은 `gpt-oss-20b` (실습용 `Qwen/Qwen2.5-14B`)를 사용한다.

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig

# 1. 모델 로드 (4-bit QLoRA)
model_id = "openai/gpt-oss-20b"
#model_id = "Qwen/Qwen3-14B"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    attn_implementation="eager" # Qwen3-14B의 경우 "sdpa"
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# 2. 학습 전처리
model = prepare_model_for_kbit_training(model)

# 3. LoRA 어댑터 설정
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

print("모델 준비 완료")

MXFP4 quantization requires Triton and kernels installed: CUDA requires Triton >= 3.4.0, XPU requires Triton >= 3.5.0, we will default to dequantizing the model to bf16


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

모델 준비 완료


## 6. 학습 실행

도메인 적응 학습은 일반적인 SFT와 동일한 파이프라인을 따르지만, **과적합(Overfitting)**에 주의해야 한다. 도메인 지식을 배우려다 일반적인 대화 능력을 잃어버릴 수 있기 때문이다.

In [4]:
# 학습 설정
training_args = SFTConfig(
    output_dir="./GPT-OSS-20B-finance",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    optim="paged_adamw_8bit",      # 메모리 효율화
    logging_steps=10,
    save_strategy="steps",
    save_steps=50,
    fp16=False,
    bf16=True,
    #max_seq_length=512,
    dataset_text_field="text",
)

# Trainer 생성
trainer = SFTTrainer(
    model=model,
    train_dataset=formatted_dataset,
    args=training_args,
    processing_class=tokenizer,
)

print("금융 도메인 적응 학습 시작...")
trainer.train()
print("학습 완료!")

# 모델 저장
trainer.save_model("./GPT-OSS-20B-financ-final")

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': 199998, 'pad_token_id': 200002}.


금융 도메인 적응 학습 시작...


wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kubwai to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,3.090000
20,2.406200
30,2.354000
40,2.276200
50,2.255200
60,2.264800


학습 완료!


## 7. 도메인 적응 핵심 전략

성공적인 도메인 모델을 만들기 위해 반드시 고려해야 할 4가지 전략이다.

1. **전문 데이터셋 수집**: 인터넷 긁어모으기(Crawling)보다는 교과서, 논문, 사내 문서 등 검증된 데이터를 사용해야 한다. 가능하다면 도메인 전문가(SME)의 검수를 거치는 것이 좋다.
2. **시스템 프롬프트 활용**: 모델에게 역할을 명확히 부여하면 말투와 전문 용어 사용 빈도가 달라진다.
3. **용어 일관성 (Terminology)**: 같은 개념을 다르게 부르지 않도록 데이터셋 내에서 용어를 통일해야 한다.
4. **면책 조항 (Disclaimer)**: 특히 의료(Medical)나 법률(Legal) 분야는 모델이 환각(Hallucination)을 일으킬 경우 치명적일 수 있다. 시스템 프롬프트에 "전문가와 상의하십시오"라는 문구를 필수적으로 포함해야 한다.

## 8. 요약

이 챕터에서는 금융 데이터셋과 시스템 프롬프트를 결합하여 모델을 금융 전문가로 튜닝하는 방법을 실습했다. 도메인 적응은 단순히 데이터를 넣는 것을 넘어, 모델의 **페르소나**를 정의하고 **전문 지식**을 체계적으로 주입하는 과정임을 이해했다.